In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import mysql.connector
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from user_agent import generate_user_agent, generate_navigator
from selenium.common.exceptions import UnexpectedAlertPresentException
import requests
import datetime

In [2]:
def movie_crawling(webdriver_path, service, driver, id):
    webdriver_path = webdriver_path
    service = service
    driver = driver
    try:
        url = f'http://cine21.com/movie/info/?movie_id={id}'
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        sub_info = soup.find_all('p', class_='sub_info')
        sub_info_list = [i.text.strip().replace('\n', ' ').replace('\t', ' ') for i in sub_info]
        ko_name = soup.find('p', class_='tit').text
        en_name = soup.find('p', class_='tit_eng').text.strip().split('\n')[-1]
        story = soup.find('div', class_='story').text.strip().replace('\t', '').replace('\n', ' ').replace('  ', ' ')
        if story == '':
            story = None
        year = sub_info_list[0].split()[0]
        country = sub_info_list[0].split()[1]
        poster_link = soup.find('div', class_='mov_poster').find('img')['src']
        if poster_link == 'https://image.cine21.com/resize//noimg[X230,330].jpg':
            poster_link = None

        try:
            if sub_info_list[2][:3] == '개봉일' and len(sub_info_list[2][6:16]) == 10:
                pd.to_datetime(sub_info_list[2][6:16])
                release_date = sub_info_list[2][6:16]
            else:
                release_date = None
        except:
            release_date = None
    
        if sub_info_list[2][17:21] == '누적관객':
            audience = int(sub_info_list[2][24:].replace(',', '').replace('명', '').replace(',', ''))
        else:
            audience = None
        
        if sub_info_list[1].split('상영시간 : ')[0] != '':
            genre = sub_info_list[1].split('상영시간 : ')[0][:-1]
            running_time = int(sub_info_list[1].split('상영시간 : ')[1][:-1])
        else:
            genre = None
            running_time = int(sub_info_list[1].split('상영시간 : ')[1][:-1])

        if sub_info_list[3][0:2] == '감독':
            director = sub_info_list[3][18:]
        else:
            director = None

        for i in range(1, 6):
            if soup.find('span', class_=f'grade00{i}') != None:
                grade = soup.find('span', class_=f'grade00{i}').text
                break
            elif i == 5:
                grade = None

        return [id, ko_name, en_name, year, country, grade, genre, running_time, release_date, audience, director, story, poster_link]
    
    except UnexpectedAlertPresentException as e1:
        return e1.alert_text
    
    except IndexError as e2:
        return '영화 정보 불충분'
    
    except AttributeError as e3:
        return '일시적으로 서비스 장애가 발생하였습니다.'

In [3]:
create_table_query = """
CREATE TABLE IF NOT EXISTS movies (
    id INT PRIMARY KEY,
    ko_name VARCHAR(255),
    en_name VARCHAR(255),
    country VARCHAR(255),
    grade VARCHAR(255),
    genre VARCHAR(255),
    runtime INT,
    release_date DATE,
    audience INT,
    director VARCHAR(255),
    story VARCHAR(255),
    poster_link VARCHAR(255))"""

In [4]:
# # My SQL 데이터베이스 연결
# conn = mysql.connector.connect(
#     host = 'localhost',
#     user = 'root',
#     password = 'signcity114',
#     database = 'MyDB'
# )

# cursor = conn.cursor()

# cursor.execute(create_table_query)
# conn.commit()
# print("테이블이 성공적으로 생성되었습니다")

# conn.close()

In [63]:
# My SQL 데이터베이스 연결
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'signcity114',
    database = 'MyDB'
)

# 연결 확인
if conn.is_connected():
    print("MySQL에 성공적으로 연결되었습니다")
else:
    print("MySQL 연결 실패")

cursor = conn.cursor()

webdriver_path = '/Users/anhyojun/WebDriver/chromedriver'
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

insert_query = """
INSERT INTO movie
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for i in range(38267, 40000):
    data = movie_crawling(webdriver_path, service, driver, i)
    if data == '존재하지 않는 영화입니다.':
        print(data)
        continue
    elif data == '성인용 컨텐츠입니다. 로그인 해 주세요.':
        print(data)
        continue
    elif data == '영화 정보 불충분':
        print(data)
        continue
    elif data == '일시적으로 서비스 장애가 발생하였습니다.':
        print(data)
        continue
    cursor.execute(insert_query, data)
    conn.commit()
    print(data)

MySQL에 성공적으로 연결되었습니다
일시적으로 서비스 장애가 발생하였습니다.
일시적으로 서비스 장애가 발생하였습니다.
일시적으로 서비스 장애가 발생하였습니다.
일시적으로 서비스 장애가 발생하였습니다.
성인용 컨텐츠입니다. 로그인 해 주세요.
일시적으로 서비스 장애가 발생하였습니다.
일시적으로 서비스 장애가 발생하였습니다.
[38274, '초한지 : 영웅의 부활', 'The Last Supper', '2013', '중국', '15세이상관람가', '역사·서사, 액션, 블록버스터', 116, '2013-03-28', 8722, '루추안', '3년의 역사적 고증 200억의 제작비 대서사 액션 블록버스터 천하를 탐낸 세 남자가 있었다! 항우와 유방, 그리고 한신…세계의 운명을 건 영웅들의 전쟁이 시작된다! 진나라에 대항하는 의군을 이끌고 있던 패왕 항우(다니엘 우)와 용의 아들로 불리며 큰 그릇을 자랑하던 유방(류예), 그리고 믿음직스러운 부하 한신(장첸). 세 남자는 의기투합하여 백성들에게 패악을 부리던 진나라를 무너뜨리고 평화를 되찾는다. 그러나 유방은 화려한 진시황의 왕궁을 보는 순간, 자신도 몰랐던 탐욕에 물든다. 피를 나눈 형제보다 가까웠던 세 남자는, 이제 천하를 갖기 위해 서로에게 칼을 겨누고 전세계의 운명을 건 최후의 전쟁을 시작하는데…', 'https://image.cine21.com/resize/cine21/poster/2013/0304/14_16_49__51342e4146724[X230,330].jpg']
일시적으로 서비스 장애가 발생하였습니다.
일시적으로 서비스 장애가 발생하였습니다.
일시적으로 서비스 장애가 발생하였습니다.
일시적으로 서비스 장애가 발생하였습니다.
[38279, '어메이징 스파이더맨2', 'The Amazing Spider-Man 2', '2013', '미국', '12세이상관람가', '블록버스터, SF', 142, '2014-04-23', 4168350, '마크 웹', '연습은 끝났다! 적이 강해진 만큼 그도 강해져야만 한다! 

WebDriverException: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=129.0.6668.101)
Stacktrace:
0   chromedriver                        0x0000000105120200 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x00000001051186e8 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x0000000104d2c260 cxxbridge1$string$len + 89488
3   chromedriver                        0x0000000104d257bc cxxbridge1$string$len + 62188
4   chromedriver                        0x0000000104d17654 cxxbridge1$string$len + 4484
5   chromedriver                        0x0000000104d1908c cxxbridge1$string$len + 11196
6   chromedriver                        0x0000000104d17a00 cxxbridge1$string$len + 5424
7   chromedriver                        0x0000000104d1727c cxxbridge1$string$len + 3500
8   chromedriver                        0x0000000104d17208 cxxbridge1$string$len + 3384
9   chromedriver                        0x0000000104d1559c core::str::slice_err

In [54]:
webdriver_path = '/Users/anhyojun/WebDriver/chromedriver'
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)
movie_crawling(webdriver_path, service, driver, 415)

[415,
 '언지프',
 'Unzipped',
 '미국',
 '15세이상관람가',
 '다큐멘터리',
 73,
 None,
 7903,
 '더글러스 키브',
 '흥미진진하고 즐거움을 주는 매우 사적인 패션 디자이너 마즈라히는 Nanlook of the North, Samantha Stevens 등 다양한 문화계의 우상들에게 영감을 얻으면서 좌절과 기쁨과 공포로 가득한 모델들이 걸어다니는 진짜 패션쇼를 만들어내기 위해 분주하다. 그는 중요한 잡지 편집자를 위해 자신의 디자인을 재검토하고 루브르에서 기록으로 남아 있는 패션 콜렉션을 보기 위해 파리로 여행하기도 한다.  쇼가 가까워지자 그는 디자인에 생명을 불어넣고, 케이트 못, 린다 에반젤리스타, 신디 크로포드 같은 세계 톱모델들과 계약을 체결하는 과정을 거치면서 유명인사들과 비평가들로 들어찬 쇼장에서 대담하고 복잡미묘하게 신경을 곤두서게 하는 쇼를 진행한다.',
 'https://image.cine21.com/resize/cine21/poster/2005/0615/M0010197_[X230,330].jpg']